In [10]:
import os
import numpy as np
import plotly.graph_objs as go
from scipy.special import ndtri

In [11]:
ccaList = ['0', '5', '6', '9']
ccaMap = ["Gcc", "Nada", "", "Fixed", 
  "Cubic", "Bbr", "Copa", "Yeah", 
  "Illinois", "Vegas", "Htcp", "Bic", 
  "LinuxReno", "Scalable", "", ""]

baseRttStart = 20000
baseRttEnd = 30000

cutInterval = 200

highRttThresVar = ndtri (0.99995) # ndtri () is the inverse cdf of gaussian. this roughly represents out of 4-sigma (1/10k)

In [12]:
def ProcessDir (cut, dirname):
    jitterDurationTrace = []
    for trace in os.listdir (dirname):
        if "cut-%d-" % cut in trace:
            cutTime = int (trace.split ('-')[2].split (".")[0])
            if cutTime % 10 != 0:
                continue
            baseRtts = []
            jitterStartTimeStamp = 0
            baseRtt = 0
            jitterDurationTrace.append (0)
            with open (os.path.join (dirname, trace, 'bw.tr')) as f:
                for line in f.readlines ():
                    timeStamp = int (line.split ()[0])
                    rtt = int (line.split ()[6])
                    if timeStamp >= baseRttStart and timeStamp <= baseRttEnd:
                        baseRtts.append (rtt)
                        continue
                    elif timeStamp > cutInterval * cutTime:
                        if not baseRtt:
                            baseRtt = np.mean (baseRtts)
                            baseRttVar = max (10, np.std (baseRtts))
                        if not jitterStartTimeStamp and rtt > baseRtt + highRttThresVar * baseRttVar:
                            jitterStartTimeStamp = timeStamp
                        elif jitterStartTimeStamp:
                            if rtt > baseRtt + highRttThresVar * baseRttVar:
                                jitterDurationTrace[-1] = timeStamp - jitterStartTimeStamp
                            else:
                                break
    
    durationTraceMean = np.mean (jitterDurationTrace)
    durationTraceStd = np.std (jitterDurationTrace)
    jitterDurationTrace = [x for x in jitterDurationTrace 
        if (x >= durationTraceMean - ndtri (0.995) * durationTraceStd and x <= durationTraceMean + ndtri (0.995) * durationTraceStd)]
    return jitterDurationTrace

In [13]:
# cca-cut exps
cutList = [2, 3, 4, 5, 6, 7, 8, 10, 12, 14, 16, 20, 25, 30, 35, 40]
queueSize = 250

jitterCca = {}
for cca in ccaList:
    jitterCca[cca] = []
    for cut in cutList:
        dirname = "ns-3.34/logs-design-cca/%s_Video1/PfifoFast_0_%d" % (cca, queueSize)
        jitterDurationTrace = ProcessDir (cut, dirname)
        jitterCca[cca].append (np.mean (jitterDurationTrace) if jitterDurationTrace else 0)

tmpStr = "k"
for cca in ccaList:
    tmpStr += "\t%s" % ccaMap[int (cca, 16)]
print (tmpStr)
for cut in cutList:
    tmpStr = "%d" % cut
    for cca in ccaList:
        tmpStr += "\t%.0f" % jitterCca[cca][cutList.index (cut)]
    print (tmpStr)         

k	Gcc	Bbr	Copa	Vegas
2	573	1108	0	40
3	1316	1528	60	160
4	2493	1928	160	280
5	3404	2768	232	368
6	4911	2196	288	2336
7	5800	2184	352	5200
8	6098	2312	432	8432
10	8020	3200	1656	14080
12	8792	3780	2020	8548
14	10356	3828	2444	7168
16	11262	4532	2868	15964
20	13596	6056	3772	21484
25	13884	8104	4592	29024
30	15249	10560	5268	18624
35	19352	14240	6572	19664
40	21068	23160	7048	9702


In [14]:
# cca-cut-rtt exps
cut = 2
rtPropList = [40, 80, 120, 160, 200]

jitterCca = {}
for cca in ccaList:
    jitterCca[cca] = []
    for rtProp in rtPropList:
        queueSize = int (rtProp * 6.25)
        dirname = "ns-3.34/logs-design-cca/%s_Video1/PfifoFast_0_%d" % (cca, queueSize)
        jitterDurationTrace = ProcessDir (cut, dirname)
        jitterCca[cca].append (np.mean (jitterDurationTrace) if jitterDurationTrace else 0)

tmpStr = "rtProp"
for cca in ccaList:
    tmpStr += "\t%s" % ccaMap[int (cca, 16)]
print (tmpStr)
for rtProp in rtPropList:
    tmpStr = "%6d" % rtProp
    for cca in ccaList:
        tmpStr += "\t%d" % jitterCca[cca][rtPropList.index (rtProp)]
    print (tmpStr)

rtProp	Gcc	Bbr	Copa	Vegas
    40	573	1108	0	40
    80	872	2380	0	137
   120	1108	3540	80	44
   160	1152	3960	124	68
   200	956	5076	116	152
